# Automatic detection of times of LoC (Loss of Consciousness) and RoC (Return of Consciousness)

**Goal**

- Load the data
- Compute the LoC and RoC 
- Visualize LoC and RoC on a spectrogram



In [ ]:
import mne

from boostlocroc.LoC_RoC import extract_loc_roc, plot_spectrogram_proba
from boostlocroc.utils import truncate_fif

### Load one EEG file

In [ ]:
# Here we load one EEG recording from the vitaldb dataset
filename = "data/VdB_fif/0009.vital_eeg.fif"  # "data/example_eeg.fif"
file_tmp = mne.io.read_raw_fif(filename)

### LoC/RoC detection 
Steps : 
1. Preprocess EEG file
2. Estimate times of LoC and Roc using a pretrained ML model
3. Plot spectrogram, predicted probabilities and classification.

**Remark**
The following code is used on an example eeg file but should be easily extended to other dataset. 

The following cell is thus only for pedagogical purposes, but can also serve as an inspection step. 

In [ ]:
%%time
# Preprocess EEG recording
file_tmp = truncate_fif(file_tmp)
sfreq = file_tmp.info["sfreq"]
duration_sec = file_tmp.n_times / sfreq

if duration_sec > 5 * 60:
    # Compute LoC and RoC
    time_loc, time_roc, t_proba, proba, loc_params, roc_params = extract_loc_roc(file_tmp)
    # Visualize Spectrogram, EEG, and probabilities
    time = file_tmp.times
    fp2 = file_tmp.get_data(picks=1).squeeze() * 10**6  # conversion volts -> microvolts
    plot_spectrogram_proba(time_loc, time_roc, fp2, sfreq, time, t_proba, proba)